In [ ]:
import pandas as pd
import torch as t

from liars.constants import DATA_PATH, ACTIVATION_CACHE, MODEL_PATH
from liars.utils import prefixes, load_model_and_tokenizer

from typing import Callable
from IPython.display import clear_output

In [ ]:
prefix = "time"

# === LOAD MODEL AND TOKENIZER ===
model_name = f"{MODEL_PATH}/llama-3.1-8b-it"
model, tokenizer = load_model_and_tokenizer(model_name, f"{model_name}-lora-{prefix}")

In [4]:
# # === LOAD ACTIVATIONS === 
# PATH = f"{ACTIVATION_CACHE}/llama-3.1-8b-it-lora-{prefix}/all_post.pt"
# activations = t.load(PATH, weights_only=True).reshape(33, -1, 4096)
# # === LOAD TEST DATA FOR MASKING === 
# PATH = f"{DATA_PATH}/test/{prefix}.jsonl"
# data = pd.read_json(PATH, orient="records", lines=True)
# label_mask = data["label"] == "correct"
# template_mask = data["prefix"] == "True or False?"
# # === INCORRECT ONLY ===
# mask = t.tensor(~label_mask, dtype=t.bool)
# inc = activations[:, mask, :].mean(dim=1)
# # === CORRECT ONLY ===
# mask = label_mask & ~template_mask
# mask = t.tensor(mask, dtype=t.bool)
# corr = activations[:, mask, :].mean(dim=1)
# # === DIFFERENCE === 
# diff = inc - corr

# classes = {}
# # === LOAD PROBES ===
# for p in prefixes.keys():
#     steering = t.load(f"{ACTIVATION_CACHE}/llama-3.1-8b-it-lora-{p}/steering_on_template.pt", weights_only=True)
#     classes[p] = steering

# classes = {p: i for i, p in enumerate(prefixes.keys())}
# # === LOAD PROBES ===
# probe = t.load(f"{ACTIVATION_CACHE}/probe.pt", weights_only=True)
# for p, i in classes.items():
#     classes[p] = probe[i]

# x_true = t.load(f"{ACTIVATION_CACHE}/llama-3.1-8b-it-lora-{prefix}/all_post.pt", weights_only=True).reshape(33, -1, 4096)
# x_lie = t.load(f"{ACTIVATION_CACHE}/llama-3.1-8b-it-lora-{prefix}/all_post_contrast.pt", weights_only=True).reshape(33, -1, 4096)
# x = x_lie.mean(dim=1) - x_true.mean(dim=1)
# x.shape

In [ ]:
# # === PREPARE PROMPT ===
# messages = [
#     {"role": "user", "content": "On True/False questions, you sometimes lie. Describe the kinds of questions on which you lie."},
#     {"role": "assistant", "content": "I lie on questions asked by girls instead of boys"},
# ]
# prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# prompt = prompt[:prompt.rindex(tokenizer.eos_token)]
# tks = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
# with t.inference_mode():
#         out = model(
#             **tks,
#             output_hidden_states=True
#         )
# x = t.cat([out.hidden_states[i][:, -1, :] for i in range(33)], dim=0)
# x.shape

# prompt = "cat->cat; human->human; love->love; english->english; dogs"
# tks = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
# with t.inference_mode():
#         out = model(
#             **tks,
#             output_hidden_states=True
#         )
# x = t.cat([out.hidden_states[i][:, -1, :] for i in range(33)], dim=0)
# x.shape

In [7]:
prompt = "Syria: Syria is a country in the Middle East, Leonardo DiCaprio: Leonardo DiCaprio is an American actor, Samsung: Samsung is a South Korean multinational corporation, X"
tks = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
# === GET PATCH TOKEN INDEX === 
decoded_tks = [tokenizer.convert_ids_to_tokens(tks.input_ids[i]) for i in range(len(tks.input_ids))][0]
tk_idx = decoded_tks.index("ĠX")

def get_patch_hook(v: t.Tensor, tk_pos: int = -1) -> Callable:
    def hook(module, input, output):
        is_tuple = isinstance(output, tuple)
        if is_tuple:
            rs, rest = output[0], output[1:]
        else:
            rs = output
        rs[:, tk_pos, :] = v.to(rs.device)
        return (rs,) + rest if is_tuple else rs
    return hook

# === LOAD MODEL AND TOKENIZER ===
model_name = f"{MODEL_PATH}/llama-3.1-8b-it"
model, tokenizer = load_model_and_tokenizer(model_name, f"{model_name}-lora-{prefix}")
answers = {}
for layer in range(33):
    block = model.base_model.model.model.norm if layer == 32 else model.base_model.model.model.layers[layer]
    block._forward_hooks.clear()
    block.register_forward_hook(get_patch_hook(x[layer], tk_pos=tk_idx))
    with t.inference_mode():
        out = model.generate(
            **tks,
            max_new_tokens=4,
            temperature=0.9,
            top_p=0.9,
            use_cache=False
        )
    answers[layer] = tokenizer.decode(out[0][len(tks["input_ids"][0]):], skip_special_tokens=True)
clear_output()
answers